#### https://barnesanalytics.com/bayesian-poisson-ab-testing-in-pymc3-on-python

In [5]:
import pandas as pd
matplotlib_style = 'fivethirtyeight' #@param ['fivethirtyeight', 'bmh', 'ggplot', 'seaborn', 'default', 'Solarize_Light2', 'classic', 'dark_background', 'seaborn-colorblind', 'seaborn-notebook']
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
import matplotlib.axes as axes;
from matplotlib.patches import Ellipse
%matplotlib inline
import seaborn as sns; sns.set_context('notebook')
from IPython.core.pylabtools import figsize
#@markdown This sets the resolution of the plot outputs (`retina` is the highest resolution)
notebook_screen_res = 'retina' #@param ['retina', 'png', 'jpeg', 'svg', 'pdf']
%config InlineBackend.figure_format = notebook_screen_res

import tensorflow as tf

import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

In [7]:
df = pd.read_csv("/home/tbrownex/data/Bayes/HorseKicks.csv")

In [4]:
# Model the count by year as a Possion distribution so need to figure out Lamba for each corps
# Set initial value of Lambda as the inverse of the average across all the data
tmp = df.copy(deep=True)
tmp.drop(columns="Year", inplace=True)
alpha = 1.0 /np.array(tmp).mean()

In [5]:
df.index=df["Year"]
df.drop(columns="Year", inplace=True)

In [ ]:
def joint_log_prob(data, alpha):
    tfd = tfp.distributions
    rvLambda       = tfd.Exponential(rate=alpha)
    rvObservation = tfd.Poisson(rate=rvLambda)
    return (rvLambda_1.log_prob(Lambda) + tf.reduce_sum(rvObservation.log_prob(data)))

In [10]:
def process(data, alpha):
    with mc.Model() as model:
        Lambda      = mc.Exponential("lambda", alpha)
        observation = mc.Poisson(rate=Lambda)
        return ( tf.reduce_sum(observation.log_prob(data)))

In [5]:
# Turn the dataframe column into a Tensor with shape [-1,]
def formatData(df, col):
    return tf.convert_to_tensor(value=df[col].values,
                                dtype=tf.float32,
                                name="colData")

In [6]:
alpha = np.array(1. / data.mean(), np.float32)

for col in df.columns:
    data = formatData(df, col)
    process(data, alpha)
    print("done")
    break

done


In [ ]:
# Set the chain's start state.
initial_chain_state = [
    tf.cast(tf.reduce_mean(count_data), tf.float32) * tf.ones([], dtype=tf.float32, name="init_lambda1"),
    tf.cast(tf.reduce_mean(count_data), tf.float32) * tf.ones([], dtype=tf.float32, name="init_lambda2", tf.float32),
    0.5 * tf.ones([], dtype=tf.float32, name="init_tau"),
]


# Since HMC operates over unconstrained space, we need to transform the
# samples so they live in real-space.
unconstraining_bijectors = [
    tfp.bijectors.Exp(),       # Maps a positive real to R.
    tfp.bijectors.Exp(),       # Maps a positive real to R.
    tfp.bijectors.Sigmoid(),   # Maps [0,1] to R.  
]


def joint_log_prob(count_data, lambda_1, lambda_2, tau):
    tfd = tfp.distributions
 
    alpha = (1. / tf.reduce_mean(count_data))
    rv_lambda_1 = tfd.Exponential(rate=alpha)
    rv_lambda_2 = tfd.Exponential(rate=alpha)
 
    rv_tau = tfd.Uniform()
 

    lambda_ = tf.gather(
         [lambda_1, lambda_2],
         indices=tf.to_int32(tau * tf.cast(tf.size(count_data), tf.float32) <= tf.cast(tf.range(tf.size(count_data)), tf.float32)))
    rv_observation = tfd.Poisson(rate=lambda_)
 
    return (
         rv_lambda_1.log_prob(lambda_1)
         + tf.reduce_sum(rv_observation.log_prob(count_data))
    )

In [ ]:
# Define a closure over our joint_log_prob.
def unnormalized_log_posterior(lambda1, lambda2, tau):
    return joint_log_prob(count_data, lambda1, lambda2, tau)

In [ ]:
# Initialize the step_size. (It will be automatically adapted.)
with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    step_size = tf.get_variable(
        name='step_size',
        initializer=tf.constant(0.05, dtype=tf.float32),
        trainable=False,
        use_resource=True
    )

In [ ]:
# Sample from the chain.
[lambda_samples,], kernel_results = tfp.mcmc.sample_chain(
    num_results=100000,
    num_burnin_steps=10000,
    current_state=initial_chain_state,
    kernel=tfp.mcmc.TransformedTransitionKernel(
        inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=unnormalized_log_posterior,
            num_leapfrog_steps=2,
            step_size=step_size,
            step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(),
            state_gradients_are_stopped=True),
        bijector=unconstraining_bijectors))

# tau_samples, lambda_1_samples, lambda_2_samples contain
# N samples from the corresponding posterior distribution
N = tf.shape(lambda_samples)[0]

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    [lambda_samples_,kernel_results_,N_,texts_per_day_,] = sess.run([lambda_samples,
                                                                     kernel_results,
                                                                     N,
                                                                     expected_texts_per_day,])
print("acceptance rate: {}".format(
    kernel_results_.inner_results.is_accepted.mean()))
print("final step size: {}".format(
    kernel_results_.inner_results.extra.step_size_assign[-100:].mean()))